In [19]:
import folium
import pandas as pd

# Thay đổi đường dẫn đến tệp CSV thực tế
csv_file = 'D:/study/Data_visualize/CK/air_quality/Khu vực.csv'

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv(csv_file)

# Tạo một bản đồ
m = folium.Map(location=[14.0583, 108.2772], zoom_start=6)  # Tọa độ trung tâm Việt Nam

# Đánh dấu các địa điểm từ dữ liệu CSV lên bản đồ
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], tooltip=row['Địa điểm Trạm đo']).add_to(m)
# Hiển thị bản đồ
m


In [4]:
import os
import pandas as pd
import tkinter as tk
from tkcalendar import Calendar
import folium

# Đọc dữ liệu từ tệp CSV chứa thông tin vị trí
location_csv_file = 'D:/study/Data_visualize/CK/air_quality/Khu vực.csv'
df_location = pd.read_csv(location_csv_file)

# Đọc dữ liệu từ các tệp CSV chứa thông tin mức độ ô nhiễm
data_folder = 'D:/study/Data_visualize/CK/air_quality/data/'

# Khởi tạo một danh sách để chứa các DataFrame
data_frames = []

# Lặp qua tất cả các tệp trong thư mục và đọc chúng vào danh sách
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        data_frames.append(df)

# Nối tất cả các DataFrame lại với nhau để tạo một DataFrame lớn
all_data = pd.concat(data_frames, ignore_index=True)

def show_map():
    selected_date = date_cal.get_date()  # Lấy ngày được chọn từ lịch

    selected_date_str = pd.to_datetime(selected_date, format='%m/%d/%y').strftime('%Y-%m-%d')


    # Tạo một bản đồ
    m = folium.Map(location=[14.0583, 108.2772], zoom_start=6)  # Tọa độ trung tâm Việt Nam

    # Đánh dấu các địa điểm từ dữ liệu CSV lên bản đồ
    for index, row in df_location.iterrows():
        folium.Marker([row['latitude'], row['longitude']], tooltip=row['Địa điểm Trạm đo']).add_to(m)

    # Đánh dấu mức độ ô nhiễm trên bản đồ
    selected_data = all_data[all_data['date'] == selected_date_str]
    for index, row in selected_data.iterrows():
        location = row['địa điểm trạm đo']
        pollution_level = row['mức độ ô nhiễm']

        location_info = df_location[df_location['Địa điểm Trạm đo'] == location]
        if not location_info.empty:
            lat, lon = location_info.iloc[0]['latitude'], location_info.iloc[0]['longitude']
            color = 'green'  # Mặc định cho mức độ ô nhiễm 'Tốt'

            # Đặt màu cho các mức ô nhiễm khác nhau
            if pollution_level == 'Vừa phải':
                color = 'lightblue'
            elif pollution_level == 'Không tốt cho người nhạy cảm':
                color = 'orange'
            elif pollution_level == 'Không tốt':
                color = 'red'
            elif pollution_level == 'Rất không tốt':
                color = 'darkpurple'
            elif pollution_level == 'Nguy hiểm':
                color = 'darkred'

            folium.Marker([lat, lon], icon=folium.Icon(color=color),
                          popup=f'{location}<br>Mức độ ô nhiễm: {pollution_level}').add_to(m)


app = tk.Tk()
app.title("Mức độ ô nhiễm không khí")

# Tạo lịch để chọn ngày
date_cal = Calendar(app)
date_cal.pack()

# Tạo nút để hiển thị bản đồ cho ngày đã chọn
show_button = tk.Button(app, text="Hiển thị Bản đồ", command=show_map)
show_button.pack()

app.mainloop()


In [8]:
import os
import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson

# Đọc dữ liệu từ tệp CSV chứa thông tin vị trí
location_csv_file = 'D:/study/Data_visualize/CK/air_quality/Khu vực.csv'
df_location = pd.read_csv(location_csv_file)

# Đọc dữ liệu từ các tệp CSV chứa thông tin mức độ ô nhiễm
data_folder = 'D:/study/Data_visualize/CK/air_quality/data/'

# Khởi tạo một danh sách để chứa các DataFrame
data_frames = []

# Lặp qua tất cả các tệp trong thư mục và đọc chúng vào danh sách
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        data_frames.append(df)

# Nối tất cả các DataFrame lại với nhau để tạo một DataFrame lớn
all_data = pd.concat(data_frames, ignore_index=True)

# Tạo một danh sách các ngày có sẵn trong dữ liệu và sắp xếp chúng
available_dates = all_data['date'].unique()
available_dates.sort()

# Tính toán khoảng thời gian giữa các ngày
date_diff = (pd.to_datetime(available_dates[-1]) - pd.to_datetime(available_dates[0])).days

# Tạo một bản đồ
m = folium.Map(location=[14.0583, 108.2772], zoom_start=6)

# Tạo dữ liệu dạng GeoJSON
features = []
for date in available_dates:
    selected_data = all_data[all_data['date'] == date]
    for index, row in df_location.iterrows():
        lat, lon = row['latitude'], row['longitude']
        location_data = selected_data[selected_data['địa điểm trạm đo'] == row['Địa điểm Trạm đo']]
        if not location_data.empty:
            aqi = location_data.iloc[0]['aqi']
            color = 'green'
            if aqi > 70:
                color = 'red'
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [lon, lat]
                },
                "properties": {
                    "time": date,
                    "style": {
                        "color": color,
                        "icon": "circle",
                        "iconstyle": {
                            "fillColor": color,
                            "fillOpacity": 0.8,
                            "radius": 8
                        }
                    }
                }
            }
            features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# Tạo biểu đồ dạng chạy theo thời gian với dữ liệu dạng GeoJSON
TimestampedGeoJson(geojson_data, period="P1D", add_last_point=True).add_to(m)

# Lưu sơ đồ ra một tệp HTML
m.save('air_quality_time_series.html')
